<a href="https://colab.research.google.com/github/dhruvakmit/tfdlvs/blob/main/20BD1A6609.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os


In [ ]:
data=pd.read_csv('/content/housing.csv')

In [ ]:
data


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [ ]:
data.shape

(20640, 10)

In [ ]:
data.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

Here we are going to begin preprocessing steps,first we are going to find null values and check if they are going to affect our model. If they do affect it, we are going to drop the columns or fill them with mean values.

In [ ]:
data.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

There are 207 null values in the column total_bedrooms.

Before dropping null values, I want to check if there is any relation between total_bedrooms and median_house_value.

In [ ]:
data.corr()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
longitude,1.000000,-0.924664,-0.108197,0.044568,0.069608,0.099773,0.055310,-0.015176,-0.045967
latitude,-0.924664,1.000000,0.011173,-0.036100,-0.066983,-0.108785,-0.071035,-0.079809,-0.144160
housing_median_age,-0.108197,0.011173,1.000000,-0.361262,-0.320451,-0.296244,-0.302916,-0.119034,0.105623
total_rooms,0.044568,-0.036100,-0.361262,1.000000,0.930380,0.857126,0.918484,0.198050,0.134153
total_bedrooms,0.069608,-0.066983,-0.320451,0.930380,1.000000,0.877747,0.979728,-0.007723,0.049686
population,0.099773,-0.108785,-0.296244,0.857126,0.877747,1.000000,0.907222,0.004834,-0.024650
households,0.055310,-0.071035,-0.302916,0.918484,0.979728,0.907222,1.000000,0.013033,0.065843
median_income,-0.015176,-0.079809,-0.119034,0.198050,-0.007723,0.004834,0.013033,1.000000,0.688075
median_house_value,-0.045967,-0.144160,0.105623,0.134153,0.049686,-0.024650,0.065843,0.688075,1.000000


As we can see the correlation between median_house_value and total_bedrooms is low. So I will not be considering them for the model.

But there is a high correlation factor between median_house_value and the following columns: Housing_median_age, median_income and total_rooms.
Hence I will be considering those columns.

In [ ]:
data.corr()[['median_income','median_house_value','housing_median_age','total_rooms']]


,median_income,median_house_value,housing_median_age,total_rooms
longitude,-0.015176,-0.045967,-0.108197,0.044568
latitude,-0.079809,-0.144160,0.011173,-0.036100
housing_median_age,-0.119034,0.105623,1.000000,-0.361262
total_rooms,0.198050,0.134153,-0.361262,1.000000
total_bedrooms,-0.007723,0.049686,-0.320451,0.930380
population,0.004834,-0.024650,-0.296244,0.857126
households,0.013033,0.065843,-0.302916,0.918484
median_income,1.000000,0.688075,-0.119034,0.198050
median_house_value,0.688075,1.000000,0.105623,0.134153


In [ ]:
x=data[['median_income','housing_median_age','total_rooms']]
y=data['median_house_value'].values.reshape(-1,1)


In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
(xtrain.shape,ytrain.shape)

((16512, 3), (16512, 1))

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

We are going to try different models with different activation functions and different optimizers.

First, I am going to use relu as an activation function along with adam optimizer.

In [ ]:
model=Sequential()
model.add(Dense(50,activation='relu',input_dim=3))
model.add(Dense(150,activation='relu'))
model.add(Dense(200,activation='relu'))
model.add(Dense(1,activation='softmax'))
model.summary()
model.compile(loss='mean_absolute_error', optimizer='adam',metrics=['mean_absolute_error'])
model.fit(xtrain,ytrain,epochs=25)

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_86 (Dense)            (None, 50)                200       
                                                                 
 dense_87 (Dense)            (None, 150)               7650      
                                                                 
 dense_88 (Dense)            (None, 200)               30200     
                                                                 
 dense_89 (Dense)            (None, 1)                 201       
                                                                 
Total params: 38,251
Trainable params: 38,251
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
516/516 [==============================] - 2s 3ms/step - loss: 207193.6719 - mean_absolute_error: 207193.6719
Epoch 2/25
516/516 [==============================] - 1s 2ms

I am going to use relu as an activation function along with SGD optimizer.

In [ ]:
model1=Sequential()
model1.add(Dense(50,activation='relu',input_dim=3))
model1.add(Dense(150,activation='relu'))
model1.add(Dense(200,activation='relu'))
model1.add(Dense(1,activation='softmax'))
model1.summary()
model1.compile(loss='mean_absolute_error', optimizer='SGD',metrics=['mean_absolute_error'])
model1.fit(xtrain,ytrain,epochs=25)

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_90 (Dense)            (None, 50)                200       
                                                                 
 dense_91 (Dense)            (None, 150)               7650      
                                                                 
 dense_92 (Dense)            (None, 200)               30200     
                                                                 
 dense_93 (Dense)            (None, 1)                 201       
                                                                 
Total params: 38,251
Trainable params: 38,251
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
516/516 [==============================] - 1s 2ms/step - loss: 207193.7188 - mean_absolute_error: 207193.7188
Epoch 2/25
516/516 [==============================] - 1s 2ms

I am going to use relu as an activation function along with RMSprop optimizer.

In [ ]:
model2=Sequential()
model2.add(Dense(50,activation='relu',input_dim=3))
model2.add(Dense(150,activation='relu'))
model2.add(Dense(200,activation='relu'))
model2.add(Dense(1,activation='softmax'))
model2.summary()
model2.compile(loss='mean_absolute_error', optimizer='RMSprop',metrics=['mean_absolute_error'])
model2.fit(xtrain,ytrain,epochs=25)

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_94 (Dense)            (None, 50)                200       
                                                                 
 dense_95 (Dense)            (None, 150)               7650      
                                                                 
 dense_96 (Dense)            (None, 200)               30200     
                                                                 
 dense_97 (Dense)            (None, 1)                 201       
                                                                 
Total params: 38,251
Trainable params: 38,251
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
516/516 [==============================] - 2s 2ms/step - loss: 207193.8438 - mean_absolute_error: 207193.8438
Epoch 2/25
516/516 [==============================] - 1s 2ms

I am going to change the activation function to linear and try different optimizers for the models.

I am going to use linear as an activation function along with adam optimizer.

In [ ]:
model3=Sequential()
model3.add(Dense(50,activation='linear',input_dim=3))
model3.add(Dense(150,activation='linear'))
model3.add(Dense(200,activation='linear'))
model3.add(Dense(1,activation='softmax'))
model3.summary()
model3.compile(loss='mean_absolute_error', optimizer='adam',metrics=['mean_absolute_error'])
model3.fit(xtrain,ytrain,epochs=25)

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_98 (Dense)            (None, 50)                200       
                                                                 
 dense_99 (Dense)            (None, 150)               7650      
                                                                 
 dense_100 (Dense)           (None, 200)               30200     
                                                                 
 dense_101 (Dense)           (None, 1)                 201       
                                                                 
Total params: 38,251
Trainable params: 38,251
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
516/516 [==============================] - 2s 2ms/step - loss: 207193.7031 - mean_absolute_error: 207193.7031
Epoch 2/25
516/516 [==============================] - 1s 2ms

I am going to use linear as an activation function along with SGD optimizer.

In [ ]:
model4=Sequential()
model4.add(Dense(50,activation='linear',input_dim=3))
model4.add(Dense(150,activation='linear'))
model4.add(Dense(200,activation='linear'))
model4.add(Dense(1,activation='softmax'))
model4.summary()
model4.compile(loss='mean_absolute_error', optimizer='SGD',metrics=['mean_absolute_error'])
model4.fit(xtrain,ytrain,epochs=25)

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_102 (Dense)           (None, 50)                200       
                                                                 
 dense_103 (Dense)           (None, 150)               7650      
                                                                 
 dense_104 (Dense)           (None, 200)               30200     
                                                                 
 dense_105 (Dense)           (None, 1)                 201       
                                                                 
Total params: 38,251
Trainable params: 38,251
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
516/516 [==============================] - 1s 2ms/step - loss: 207193.7500 - mean_absolute_error: 207193.7500
Epoch 2/25
516/516 [==============================] - 1s 2ms

I am going to use linear as an activation function along with RMSProp optimizer.

In [ ]:
model5=Sequential()
model5.add(Dense(50,activation='linear',input_dim=3))
model5.add(Dense(150,activation='linear'))
model5.add(Dense(200,activation='linear'))
model5.add(Dense(1,activation='softmax'))
model5.summary()
model5.compile(loss='mean_absolute_error', optimizer='RMSprop',metrics=['mean_absolute_error'])
model5.fit(xtrain,ytrain,epochs=25)

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_106 (Dense)           (None, 50)                200       
                                                                 
 dense_107 (Dense)           (None, 150)               7650      
                                                                 
 dense_108 (Dense)           (None, 200)               30200     
                                                                 
 dense_109 (Dense)           (None, 1)                 201       
                                                                 
Total params: 38,251
Trainable params: 38,251
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
516/516 [==============================] - 2s 2ms/step - loss: 207193.8281 - mean_absolute_error: 207193.8281
Epoch 2/25
516/516 [==============================] - 1s 2ms

We have compiled 6 different models with various optimizer and activation function.

We will evaluate these models with test datasets and check the error in each model.

In [ ]:
model.evaluate(xtest,ytest)

129/129 [==============================] - 0s 1ms/step - loss: 205499.2969 - mean_absolute_error: 205499.2969


[205499.296875, 205499.296875]

In [ ]:
test=pd.DataFrame({'median_income':[5.3354],'housing_median_age':[14.0],'total_rooms':[300.0]})
test

,median_income,housing_median_age,total_rooms
0,5.3354,14.0,300.0


In [ ]:
model.predict(test)


1/1 [==============================] - 0s 53ms/step


array([[1.]], dtype=float32)